Let's go.

In [ ]:
import pandas as pd

In [ ]:
def openData():
    df=pd.read_csv("../data.csv")
    return df
house=openData()
house.info()
house['age'] = (1403-house['buildYear']) # adding age for better corr

just working a bit with the dataset to understand what's going on. you can pass this.

In [ ]:
print(house.describe())

In [ ]:
import matplotlib.pyplot as plt
house.hist()
# plt.show()

**making the test set**

In [ ]:
from zlib import crc32
import numpy as np
def is_id_in_test_set(identifier, test_ratio):
    return crc32(np.int64(identifier)) < test_ratio * 2**32
def split_data_with_id_hash(data, test_ratio, id_column):
    ids = data[id_column]
    in_test_set = ids.apply(lambda id_: is_id_in_test_set(id_, test_ratio))
    return data.loc[~in_test_set], data.loc[in_test_set]


adding an index to data as it's needed.

In [ ]:
house_with_id = house.reset_index() # adds an `index` column
train_set, test_set = split_data_with_id_hash(house_with_id, 0.2, "index")

stratified sampling:

In [ ]:
house["meter_cat"] = pd.cut(house["meter"],
bins=[0., 40, 80, 120, 160, 200., np.inf],
labels=[1, 2, 3, 4, 5, 6])
house["meter_cat"].value_counts().sort_index().plot.bar(rot=0, grid=True)
plt.xlabel("Income category")
plt.ylabel("Number of districts")
# plt.show()

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
stratSplits = []
for trainIndex, testIndex in splitter.split(house, house["meter_cat"]):
    stratTrainSetN = house.iloc[trainIndex]
    stratTestSetN = house.iloc[testIndex]
    stratSplits.append([stratTrainSetN, stratTestSetN])
    
stratTrainSet, stratTestSet = stratSplits[0]
print(stratTestSet["meter_cat"].value_counts() / len(stratTestSet))

In [ ]:
stratTrainSet, stratTestSet = stratSplits[0]
print(stratTestSet["meter_cat"].value_counts() / len(stratTestSet))

In [ ]:
for set_ in ( stratTrainSet ,stratTestSet):
    set_.drop("meter_cat", axis=1 , inplace=True)

_**START of the MODEL pre-process**_

In [ ]:
house = stratTrainSet.copy()
corr_matrix = house.corr()
print(corr_matrix['price'].sort_values(ascending=False))

_note 1_ : wow it seems that meter is really important!

In [ ]:
house = stratTrainSet.drop('price',axis=1)
labels = stratTestSet['price'].copy()

imputation

In [38]:
def imputeFrequent(cols,h):
    for col in cols:
        frequent = house[col].mode()
        h[[col]]=h[[col]].fillna(float(frequent),inplace=False)
    return h
house = imputeFrequent(['floor','rooms','meter','buildYear'], house)
house.describe()


            meter    buildYear       rooms       floor         age
count  748.000000   748.000000  746.000000  748.000000  748.000000
mean    96.050802  1393.288770    1.941019    3.687166    9.711230
std     52.553894     7.995449    0.732198    3.216173    7.995449
min      1.000000  1370.000000    1.000000    1.000000    0.000000
25%     62.000000  1386.750000    1.000000    2.000000    2.000000
50%     82.500000  1395.000000    2.000000    3.000000    8.000000
75%    115.000000  1401.000000    2.000000    4.000000   16.250000
max    650.000000  1403.000000    4.000000   27.000000   33.000000
            meter    buildYear       rooms       floor         age
count  748.000000   748.000000  748.000000  748.000000  748.000000
mean    96.050802  1393.288770    1.941176    3.687166    9.711230
std     52.553894     7.995449    0.731223    3.216173    7.995449
min      1.000000  1370.000000    1.000000    1.000000    0.000000
25%     62.000000  1386.750000    1.000000    2.000000    2.00

*Handling district*

In [ ]:
from sklearn.preprocessing import OneHotEncoder
houseCategory=house[['district']]
housingOneHot = OneHotEncoder().fit_transform(houseCategory)
housingOneHot.toarray()

number attributes:

In [ ]:
from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()
houseNum=house.select_dtypes(np.number)
housingNumStd = std_scaler.fit_transform(houseNum)

from sklearn.preprocessing import FunctionTransformer
log_transformer = FunctionTransformer(np.log, inverse_func=np.exp)
log_pop = log_transformer.transform(house[["meter"]])

**And finally The PIPELINE**

In [ ]:
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

num_pipeline = Pipeline([
("impute", SimpleImputer(strategy='most_frequent')),
("standardize", StandardScaler()),
])
houseNumPrepared=num_pipeline.fit_transform(houseNum)
cat_pipeline = make_pipeline(
SimpleImputer(strategy="most_frequent"),
OneHotEncoder(handle_unknown="ignore"))

preprocessing = ColumnTransformer([
    ('cat',cat_pipeline,['district'])
],remainder=num_pipeline)
housing_prepared = preprocessing.fit_transform(house)
house.describe()

**Train**

In [ ]:
from sklearn.linear_model import LinearRegression
lin_reg = make_pipeline(preprocessing, LinearRegression())
lin_reg.fit(house, labels)